In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import rasterio

In [3]:
nlcd = r"F:\onedrive\OneDrive - University of Central Florida\NLCD\Annual_NLCD_LndCov_2020_CU_C1V0.tif"

In [4]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject

In [5]:
target_crs = "EPSG:4326"
def reproject_raster(input_path, output_path, target_crs):
    # Open the input raster file
    with rasterio.open(input_path) as src:
        # Calculate the transform and dimensions for the target CRS
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        
        # Update metadata with new CRS and dimensions
        metadata = src.meta.copy()
        metadata.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        # Create the output raster file with the new CRS
        with rasterio.open(output_path, 'w', **metadata) as dst:
            # Reproject the data
            for i in range(1, src.count + 1):  # Loop over each band
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs
                )


In [6]:
output_path1 = r"F:\onedrive\OneDrive - University of Central Florida\NLCD\florida_nlcd_proj.tif"
reproject_raster(nlcd, output_path1, target_crs)

In [27]:
with rasterio.open(output_path1) as src:
    print(src.crs)
    print(src.bounds)


EPSG:4326
BoundingBox(left=-129.27731989810934, bottom=21.805036235947423, right=-63.118461597579426, top=52.9217197338581)


In [21]:
if gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)
print(gdf.crs)


EPSG:4326


In [11]:
import rasterio
import numpy as np

output_path1 = r"F:\onedrive\OneDrive - University of Central Florida\NLCD\florida_nlcd_proj.tif"

# Open the raster file
with rasterio.open(output_path1) as src:
    unique_values = set()  # To store unique values in the raster

    # Loop through raster blocks
    for ji, window in src.block_windows(1):  # Iterating over blocks of the first band
        # Read the data for this block
        data = src.read(1, window=window)

        # Add unique values from this block to the set
        unique_values.update(np.unique(data))

    # Convert the set to a sorted list (optional)
    print(sorted(unique_values))


[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 52, 71, 81, 82, 90, 95, 250]


In [30]:
import numpy as np
import rasterio
from rasterio.mask import mask

reclass_map = {
    11: 1, 12: 1,   # Water
    21: 2, 22: 2, 23: 2, 24: 2,  # Developed
    31: 3,   # Barren
    41: 4, 42: 4, 43: 4,  # Forest
    52: 5,   # Shrubland
    71: 6,   # Grassland
    81: 7,  # Pasture
    82: 8,  # Agriculture
    90: 9, 95: 9,  # Wetlands
}
output_path1 = r"F:\onedrive\OneDrive - University of Central Florida\NLCD\florida_nlcd_proj.tif"
output_path = r"F:\onedrive\OneDrive - University of Central Florida\NLCD\reclassified_florida_nlcd.tif"
gdf = gpd.read_file(r"F:\onedrive\OneDrive - University of Central Florida\Florida_shp\Florida_county.shp")
nodata_value = 250  # Set the no-data value
with rasterio.open(output_path1) as src:
    geom = [gdf.geometry.unary_union]  # Geometry for masking
    out_image, out_transform = mask(src, geom, crop=True)
    
    # Reclassify raster while preserving no-data value (250)
    reclassified = np.where(out_image[0] == nodata_value, nodata_value,  # Preserve no-data pixels as 250
                            np.vectorize(lambda x: reclass_map.get(x, 0))(out_image[0]))  # Apply reclassification

    # Update metadata
    out_meta = src.meta.copy()
    out_meta.update({
        'driver': 'GTiff',
        'height': reclassified.shape[0],
        'width': reclassified.shape[1],
        'transform': out_transform,
        'dtype': 'uint8',
        'nodata': nodata_value  # Set the no-data value in metadata
    })

    # Save the reclassified raster with updated metadata
    with rasterio.open(output_path, "w", **out_meta) as dst:
        dst.write(reclassified, 1)


In [33]:

# Open the reclassified raster
with rasterio.open(output_path) as src:
    data = src.read(1)  # Read the first band
    nodata = src.nodata  # Get the no-data value

# Mask out no-data values
valid_pixels = data[data != nodata]

# Compute unique class counts
unique, counts = np.unique(valid_pixels, return_counts=True)

# Compute class ratios
total_valid = counts.sum()
class_ratios = {cls: (count / total_valid)*100 for cls, count in zip(unique, counts)}

# Print class ratios
for cls, ratio in class_ratios.items():
    print(f"Class {cls}: {ratio:.2f}")

250.0
Class 1: 16.71
Class 2: 14.70
Class 3: 0.61
Class 4: 15.55
Class 5: 2.40
Class 6: 2.39
Class 7: 10.90
Class 8: 5.29
Class 9: 31.45


In [ ]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask


# Reproject to EPSG:3086 if needed
if gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)

# Open raster
with rasterio.open(output_path) as src:
    results = []

    for _, county in gdf.iterrows():
        county_name = county["NAME"]  # Adjust this field based on actual column name
        geom = [county.geometry]  # Convert to list format for rasterio.mask

        # Mask the raster with the county geometry
        try:
            out_image, out_transform = mask(src, geom, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({
            'driver': 'GTiff',
            'height': out_image.shape[1],
            'width' : out_image.shape[2],
            'transform': out_transform     
            })
            data = out_image[0]  # Extract band 1
        except Exception as e:
            print(f"Skipping {county_name}: {e}")
            continue

        # Remove no-data values
        nodata = src.nodata
        valid_pixels = data[data != nodata]

        if valid_pixels.size > 0:
            # Compute unique class counts
            unique, counts = np.unique(valid_pixels, return_counts=True)
            total_valid = counts.sum()

            # Compute class ratios
            class_ratios = {cls: count / total_valid for cls, count in zip(unique, counts)}

            # Store results
            results.append({"County": county_name, "Class Ratios": class_ratios})
        else: print(county_name)

# Print results
for res in results:
    print(f"County: {res['County']}")
    for cls, ratio in res["Class Ratios"].items():
        print(f"  Class {cls}: {ratio:.4f}")


In [38]:
result = pd.DataFrame(results)

In [ ]:
# Convert the dictionary in each row to separate columns
df_expanded = pd.json_normalize(result['Class Ratios'])

# Optionally, concatenate this expanded DataFrame back to the original DataFrame
result = pd.concat([result, df_expanded], axis=1)

# Drop the original dictionary column if it's no longer needed
result.drop(columns=['Class Ratios'], inplace=True)


In [45]:
result.to_csv(r"F:\onedrive\OneDrive - University of Central Florida\NLCD\florida_nlcd.csv", index = False)